# A Simple Resume Generator

In [1]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field


class Education(BaseModel):
    institution: str = Field(..., description="institution name")
    course: str = Field(..., description="course or degree name")
    location: Optional[str]
    from_dt: Optional[str]
    to_dt: Optional[str]


class Skill(BaseModel):
    name: str = Field(..., description="skill name")
    proficiency: Optional[str] = Field(enum=["beginner", "intermediate", "expert"])


class Project(BaseModel):
    name: str = Field(..., description="project name")
    skills: List[Skill] = Field(..., description="list of all skills required to develop this project")
    description: Optional[str] = Field(description="description of project")
    from_dt: Optional[str]
    to_dt: Optional[str]


class Experience(BaseModel):
    company: str = Field(..., description="name of company")
    designation: str = Field(...)
    location: Optional[str]
    from_dt: Optional[str]
    to_dt: Optional[str]


class ResumeData(BaseModel):
    profile: str = Field(..., description="the sentence that describes the employee")
    skills: List[Skill] = Field(..., description="list of skills")
    education: Optional[List[Education]] = Field(description="list of education")
    experience: Optional[List[Experience]] = Field(description="list of professional experiences")
    projects: Optional[List[Project]] = Field(description="list of projects")

In [2]:
import re
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.chains.structured_output.base import create_structured_output_runnable


llm = ChatOpenAI(temperature=1).with_config(config={'callbacks': [ConsoleCallbackHandler()]})
structured_llm = create_structured_output_runnable(
    ResumeData,
    llm,
    mode="openai-tools",
    enforce_function_usage=True,
    return_single=True
)
resume_gen_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a good professional resume generator"),
        ("human", "create a resume for me using the details given below, also try to describe the projects using the information given and add some sentences on your own if needed but do not assume dates and years on your own. Also rephrase the sentences to sound professional. Details:\n\nname: {name}\nskills: {skills}\nprofessional experience: {experience}\neducation: {education}\nprojects: {projects}"),
    ]
)

def parse(resume_data: ResumeData) -> str:
    return resume_data.json()

resume_gen_chain = (resume_gen_prompt | structured_llm | parse)

In [3]:
# testing chain1
resume_gen_chain.invoke({
    "name": "Ashish Agarwal",
    "skills": "python, dart, php, java, javascript",
    "experience": "senior software developer at Capgemini. since june 2017, computer vision engineer at TCS from may 2013 to march 2017, QA Tester at Accenture from january 2000 to feb 2002",
    "education": "Btech in Computer Science from delhi university from 1996 to 2000",
    "projects": "FriendBook: an open source clone of facebook and NerdMatch, a dating app for nerds",
})

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: you are a good professional resume generator\nHuman: create a resume for me using the details given below, also try to describe the projects using the information given and add some sentences on your own if needed but do not assume dates and years on your own. Also rephrase the sentences to sound professional. Details:\n\nname: Ashish Agarwal\nskills: python, dart, php, java, javascript\nprofessional experience: senior software developer at Capgemini. since june 2017, computer vision engineer at TCS from may 2013 to march 2017, QA Tester at Accenture from january 2000 to feb 2002\neducation: Btech in Computer Science from delhi university from 1996 to 2000\nprojects: FriendBook: an open source clone of facebook and NerdMatch, a dating app for nerds"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [10.71s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_in

'{"profile": "Experienced Senior Software Developer with a strong background in Python, Dart, PHP, Java, and JavaScript. Skilled in developing innovative solutions and leading projects to successful completion. Recognized for expertise in computer vision and quality assurance testing.", "skills": [{"name": "Python", "proficiency": "expert"}, {"name": "Dart", "proficiency": "intermediate"}, {"name": "PHP", "proficiency": "intermediate"}, {"name": "Java", "proficiency": "intermediate"}, {"name": "JavaScript", "proficiency": "expert"}], "education": [{"institution": "Delhi University", "course": "Btech in Computer Science", "location": null, "from_dt": "1996", "to_dt": "2000"}], "experience": [{"company": "Capgemini", "designation": "Senior Software Developer", "location": null, "from_dt": "June 2017", "to_dt": null}, {"company": "TCS", "designation": "Computer Vision Engineer", "location": null, "from_dt": "May 2013", "to_dt": "March 2017"}, {"company": "Accenture", "designation": "QA Te

In [4]:
latex_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a resume generator that generates beautiful and eye catching resumes using latex format. Dont include anything other than latex code."),
    ("human", "create a formatted resume in latex using he data given below:\n\nname: {name}\nresume data: {resume_data}")
])


def parse(message: AIMessage) -> str:
    msg = re.sub(r"(.*)(```)", r"\2", message.content, flags=re.IGNORECASE)
    msg = re.sub(r"(```)(?!latex).*", r"\1", msg, flags=re.IGNORECASE)
    msg = re.sub(r"```latex([\s\S]*?)```", r"\1", msg, flags=re.IGNORECASE)
    msg = re.sub(r"(\\documentclass{article}\n)", r"\1\\usepackage{lmodern}\n", msg, flags=re.IGNORECASE)
    return msg.strip()


chain2 = (
    {"resume_data": resume_gen_chain, "name": itemgetter("name")}
    | latex_prompt
    | llm
    | parse
)

In [5]:
response = chain2.invoke(
    {
        "name": "Ashish Agarwal",
        "skills": "python, dart, php, java, javascript",
        "experience": "senior software developer at Capgemini. since june 2017, computer vision engineer at TCS from may 2013 to march 2017, QA Tester at Accenture from january 2000 to feb 2002",
        "education": "Btech in Computer Science from delhi university from 1996 to 2000",
        "projects": "FriendBook: an open source clone of facebook and NerdMatch, a dating app for nerds",
    }, 
    # config={'callbacks': [ConsoleCallbackHandler()]}
)

print(response)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: you are a good professional resume generator\nHuman: create a resume for me using the details given below, also try to describe the projects using the information given and add some sentences on your own if needed but do not assume dates and years on your own. Also rephrase the sentences to sound professional. Details:\n\nname: Ashish Agarwal\nskills: python, dart, php, java, javascript\nprofessional experience: senior software developer at Capgemini. since june 2017, computer vision engineer at TCS from may 2013 to march 2017, QA Tester at Accenture from january 2000 to feb 2002\neducation: Btech in Computer Science from delhi university from 1996 to 2000\nprojects: FriendBook: an open source clone of facebook and NerdMatch, a dating app for nerds"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_inf

In [6]:
import subprocess


with open("resume.tex", "w") as f:
    f.write(response)

proc = subprocess.Popen(['/Library/TeX/texbin/pdflatex', 'resume.tex'])
proc.communicate()

This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./resume.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/lm/lmodern.sty)
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2024basic/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/local/texlive/2024basic/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/enumitem/enumitem.sty)
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/xcolor/xcolor.sty
(/usr/local/texlive/2024basic/texmf-dist/tex/latex/graphics-cfg/col

(None, None)